# Natural Language Processing in Action

## 3. Math with words (TF-IDF vectors)

### 3.1 Bag of words

wordの出現回数をカウントしてみる。

In [17]:
from nltk.tokenize import TreebankWordTokenizer

In [19]:
sentence = "The faster Harry got to the store, the faster Harry, the faster, would get home."

In [20]:
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(sentence.lower())

In [5]:
from collections import Counter
bag_of_words = Counter(tokens)

In [6]:
bag_of_words

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

In [7]:
bag_of_words.most_common(4)

[('the', 4), ('faster', 3), (',', 3), ('harry', 2)]

与えられたドキュメント内のwordの出現回数を Term Frequency (TF) と呼ぶ。
多くの場合、ドキュメント内の単語数で割ることで正規化する。

In [8]:
times_harry_appears = bag_of_words["harry"]
num_unique_words = len(bag_of_words)
tf = times_harry_appears / num_unique_words
round(tf, 4)

0.1818

In [15]:
kite_text = "A kite is traditionally a tethered heavier-than-air craft with wing surfaces that react against the air to create lift and drag. A kite consists of wings, tethers, and anchors. Kites often have a bridle to guide the face of the kite at the correct angle so the wind can lift it. A kite's wing also may be so designed so a bridle is not needed; when kiting a sailplane for launch, the tether meets the wing at a single point. A kite may have fixed or moving anchors. Untraditionally in technical kiting, a kite consists of tether-set-coupled wing sets; even in technical kiting, though, a wing in the system is still often called the kite. The lift that sustains the kite in flight is generated when air flows around the kite's surface, producing low pressure above and high pressure below the wings. The interaction with the wind also generates horizontal drag along the direction of the wind. The resultant force vector from the lift and drag force components is opposed by the tension of one or more of the lines or tethers to which the kite is attached. The anchor point of the kite line may be static or moving (e.g., the towing of a kite by a running person, boat, free-falling anchors as in paragliders and fugitive parakites or vehicle). The same principles of fluid flow apply in liquids and kites are also used under water. A hybrid tethered craft comprising both a lighter-than-air balloon as well as a kite lifting surface is called a kytoon. Kites have a long and varied history and many different types are flown individually and at festivals worldwide. Kites may be flown for recreation, art or other practical uses. Sport kites can be flown in aerial ballet, sometimes as part of a competition. Power kites are multi-line steerable kites designed to generate large forces which can be used to power activities such as kite surfing, kite landboarding, kite fishing, kite buggying and a new trend snow kiting. Even Man-lifting kites have been made."

In [16]:
tokenizer = TreebankWordTokenizer()
tokens = tokenizer.tokenize(kite_text.lower())
tokens_counts = Counter(tokens)

NameError: name 'TreebankWordTokenizer' is not defined

In [12]:
import nltk
nltk.download("stopwords", quiet=True)

True

In [13]:
stopwords = nltk.corpus.stopwords.words("english")
tokens = [w for w in tokens if w not in stopwords]
kite_counts = Counter(tokens)

In [17]:
kite_counts.most_common(10)

[('kite', 16),
 (',', 15),
 ('kites', 8),
 ('wing', 5),
 ('lift', 4),
 ('may', 4),
 ('also', 3),
 ('kiting', 3),
 ('flown', 3),
 ('tethered', 2)]

### 3.2 Vectorizing

In [18]:
document_vector = []
doc_length = len(tokens)
for key, value in kite_counts.most_common():
    document_vector.append(value / doc_length)

In [23]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home."]
docs.append("Harry is hairy and faster than Jill.")
docs.append("Jill is not as hairy as Harry.")

doc_tokens = [sorted(tokenizer.tokenize(doc.lower())) for doc in docs]

In [24]:
len(doc_tokens[0])

17

In [25]:
all_doc_tokens = sum(doc_tokens, [])
len(all_doc_tokens)

33

In [26]:
lexicon = sorted(set(all_doc_tokens))
len(lexicon)
lexicon

[',',
 '.',
 'and',
 'as',
 'faster',
 'get',
 'got',
 'hairy',
 'harry',
 'home',
 'is',
 'jill',
 'not',
 'store',
 'than',
 'the',
 'to',
 'would']

In [27]:
from collections import OrderedDict
import copy

zero_vector = OrderedDict((token, 0) for token in lexicon)
doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    for key, value in token_counts.items():
        vec[key] = value / len(lexicon)
    doc_vectors.append(vec)

In [28]:
doc_vectors

[OrderedDict([(',', 0.05555555555555555),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.16666666666666666),
              ('get', 0.05555555555555555),
              ('got', 0.05555555555555555),
              ('hairy', 0),
              ('harry', 0.1111111111111111),
              ('home', 0.05555555555555555),
              ('is', 0),
              ('jill', 0),
              ('not', 0),
              ('store', 0.05555555555555555),
              ('than', 0),
              ('the', 0.16666666666666666),
              ('to', 0.05555555555555555),
              ('would', 0.05555555555555555)]),
 OrderedDict([(',', 0),
              ('.', 0.05555555555555555),
              ('and', 0.05555555555555555),
              ('as', 0),
              ('faster', 0.05555555555555555),
              ('get', 0),
              ('got', 0),
              ('hairy', 0.05555555555555555),
              ('harry', 0.05

#### 3.2.1 Vector spaces

NLPではコーパス中に出現する個別の単語数が次元数となる。この次元数をNLP界隈では "K" で表す。例えば、3.2の3つの例文から作られたベクトルの次元数は `K = 18` となる。

２つのベクトルの類似度を測るために、コサイン類似度を使う。

cosΘ = A・B / |A||B|

In [128]:
import math

def cosine_sim(vec1, vec2):
    vec1 = [val for val in vec1.values()]
    vec2 = [val for val in vec2.values()]
    
    dot_prod = 0
    for i, v in enumerate(vec1):
        dot_prod += v * vec2[i]
    
    mag_1 = math.sqrt(sum([x**2 for x in vec1]))
    mag_2 = math.sqrt(sum([x**2 for x in vec2]))
    
    return dot_prod / (mag_1 * mag_2)

In [7]:
v1 = {"a": 1, "b": 0}
v2 = {"a": 1, "b": 1}
cosine_sim(v1, v2)

0.7071067811865475

### 3.3 Zipf's Law

単語の出現頻度はその順位に反比例するという法則。例えば、k番目の単語であれば、1番多い単語と比較して1/k程度の頻度になる。

In [9]:
import nltk
nltk.download("brown")
from nltk.corpus import brown
brown.words()[:10]

[nltk_data] Downloading package brown to /Users/akitanak/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [10]:
brown.tagged_words()[:5]

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL')]

In [11]:
len(brown.words())

1161192

In [13]:
from collections import Counter
puncs = set((",", ".", "--", "-", "!", "?", ":", ";", "``", "''", "(", ")", "[", "]"))
word_list = (x.lower() for x in brown.words() if x not in puncs)
token_counts = Counter(word_list)
token_counts.most_common(20)

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548),
 ('for', 9489),
 ('it', 8760),
 ('with', 7289),
 ('as', 7253),
 ('his', 6996),
 ('on', 6741),
 ('be', 6377),
 ('at', 5372),
 ('by', 5306),
 ('i', 5164)]

## 3.4 Topic modeling
出現頻度は高いがコーパス全体に渡って登場するような単語はあまり新たな情報があるとは言えない。特定のドキュメントに頻繁に登場するが、コーパス全体に渡って登場するわけではない単語はそのドキュメントの特徴を表していると言える。

IDF(Inverse Document Frequency)

In [22]:
kite_text = "A kite is traditionally a tethered heavier-than-air craft with wing surfaces that react against the air to create lift and drag. A kite consists of wings, tethers, and anchors. Kites often have a bridle to guide the face of the kite at the correct angle so the wind can lift it. A kite's wing also may be so designed so a bridle is not needed; when kiting a sailplane for launch, the tether meets the wing at a single point. A kite may have fixed or moving anchors. Untraditionally in technical kiting, a kite consists of tether-set-coupled wing sets; even in technical kiting, though, a wing in the system is still often called the kite. The lift that sustains the kite in flight is generated when air flows around the kite's surface, producing low pressure above and high pressure below the wings. The interaction with the wind also generates horizontal drag along the direction of the wind. The resultant force vector from the lift and drag force components is opposed by the tension of one or more of the lines or tethers to which the kite is attached. The anchor point of the kite line may be static or moving (e.g., the towing of a kite by a running person, boat, free-falling anchors as in paragliders and fugitive parakites or vehicle). The same principles of fluid flow apply in liquids and kites are also used under water. A hybrid tethered craft comprising both a lighter-than-air balloon as well as a kite lifting surface is called a kytoon. Kites have a long and varied history and many different types are flown individually and at festivals worldwide. Kites may be flown for recreation, art or other practical uses. Sport kites can be flown in aerial ballet, sometimes as part of a competition. Power kites are multi-line steerable kites designed to generate large forces which can be used to power activities such as kite surfing, kite landboarding, kite fishing, kite buggying and a new trend snow kiting. Even Man-lifting kites have been made."
kite_history = """Kites were invented in China, where materials ideal for kite building were readily available: silk fabric for sail material; fine, high-tensile-strength silk for flying line; and resilient bamboo for a strong, lightweight framework.
The kite has been claimed as the invention of the 5th-century BC Chinese philosophers Mozi (also Mo Di) and Lu Ban (also Gongshu Ban). By 549 AD paper kites were certainly being flown, as it was recorded that in that year a paper kite was used as a message for a rescue mission. Ancient and medieval Chinese sources describe kites being used for measuring distances, testing the wind, lifting men, signaling, and communication for military operations. The earliest known Chinese kites were flat (not bowed) and often rectangular. Later, tailless kites incorporated a stabilizing bowline. Kites were decorated with mythological motifs and legendary figures; some were fitted with strings and whistles to make musical sounds while flying. From China, kites were introduced to Cambodia, Thailand, India, Japan, Korea and the western world.
After its introduction into India, the kite further evolved into the fighter kite, known as the patang in India, where thousands are flown every year on festivals such as Makar Sankranti.
Kites were known throughout Polynesia, as far as New Zealand, with the assumption being that the knowledge diffused from China along with the people. Anthropomorphic kites made from cloth and wood were used in religious ceremonies to send prayers to the gods. Polynesian kite traditions are used by anthropologists get an idea of early “primitive” Asian traditions that are believed to have at one time existed in Asia."""

In [38]:
kite_intro = kite_text.lower()
intro_tokens = tokenizer.tokenize(kite_intro)

kite_history = kite_history.lower()
history_tokens = tokenizer.tokenize(kite_history)

# ドキュメント内の単語数
intro_total = len(intro_tokens)
history_total = len(history_tokens)
print(f"intro; {intro_total}, history; {history_total}")

intro; 363, history; 295


In [39]:
# 各ドキュメントのTF(Term Frequency)を計算する
intro_tf = {}
history_tf = {}
intro_counts = Counter(intro_tokens)
intro_tf["kite"] = intro_counts["kite"] / intro_total
history_counts = Counter(history_tokens)
history_tf["kite"] = history_counts["kite"] / history_total
print(f"Term Frequency of 'kite' in intro is {intro_tf['kite']:.4f}")
print(f"Term Frequency of 'kite' in history is {history_tf['kite']:.4f}")

Term Frequency of 'kite' in intro is 0.0441
Term Frequency of 'kite' in history is 0.0203


In [47]:
intro_tf["and"] = intro_counts["and"] / intro_total
history_tf["and"] = history_counts["and"] / history_total
print(f"Term Frequency of 'and' in intro is {intro_tf['and']:.4f}")
print(f"Term Frequency of 'and' in history is {history_tf['and']:.4f}")

Term Frequency of 'and' in intro is 0.0275
Term Frequency of 'and' in history is 0.0305


In [40]:
num_docs_containing_and = 0
num_docs_containing_kite = 0
num_docs_containing_china = 0
for doc in [intro_tokens, history_tokens]:
    if "and" in doc:
        num_docs_containing_and += 1
    if "kite" in doc:
        num_docs_containing_kite += 1
    if "china" in doc:
        num_docs_containing_china += 1

In [42]:
print(f"num_docs_containing_and: {num_docs_containing_and}")
print(f"num_docs_containing_kite: {num_docs_containing_kite}")
print(f"num_docs_containing_china: {num_docs_containing_china}")

num_docs_containing_and: 2
num_docs_containing_kite: 2
num_docs_containing_china: 1


In [41]:
intro_tf["china"] = intro_counts["china"] / intro_total
history_tf["china"] = history_counts["china"] / history_total

IDFはその単語がそのドキュメントに含まれることがどれだけ稀有であるかを表すと考えることができる。ある単語が特定のドキュメントでは頻繁に現れるがコーパスの他のドキュメントではあまり現れない場合、そのドキュメントにとってその単語は重要であると考えることができる。

In [48]:
num_docs = 2
intro_idf = {}
history_idf = {}
intro_idf["and"] = num_docs / num_docs_containing_and
history_idf["and"] = num_docs / num_docs_containing_and
intro_idf["kite"] = num_docs / num_docs_containing_kite
history_idf["kite"] = num_docs / num_docs_containing_kite
intro_idf["china"] = num_docs / num_docs_containing_china
history_idf["china"] = num_docs / num_docs_containing_china

In [49]:
intro_tfidf = {}
intro_tfidf["and"] = intro_tf["and"] * intro_idf["and"]
intro_tfidf["kite"] = intro_tf["kite"] * intro_idf["kite"]
intro_tfidf["china"] = intro_tf["china"] * intro_idf["china"]

In [50]:
history_tfidf = {}
history_tfidf["and"] = history_tf["and"] * history_idf["and"]
history_tfidf["kite"] = history_tf["kite"] * history_idf["kite"]
history_tfidf["china"] = history_tf["china"] * history_idf["china"]

In [51]:
print(intro_tf)
print(intro_idf)
print(intro_tfidf)

{'kite': 0.0440771349862259, 'china': 0.0, 'and': 0.027548209366391185}
{'and': 1.0, 'kite': 1.0, 'china': 2.0}
{'and': 0.027548209366391185, 'kite': 0.0440771349862259, 'china': 0.0}


In [52]:
print(history_tf)
print(history_idf)
print(history_tfidf)

{'kite': 0.020338983050847456, 'china': 0.010169491525423728, 'and': 0.030508474576271188}
{'and': 1.0, 'kite': 1.0, 'china': 2.0}
{'and': 0.030508474576271188, 'kite': 0.020338983050847456, 'china': 0.020338983050847456}


### 3.4.1 Return of Zipf

term: t,
document: d,
corpus: D

$
\begin{align}
\mathrm{tf}(t, d) = \frac{count(t)}{count(d)}
\end{align}
$

$
\begin{align}
\mathrm{idf}(t, D) = \log \frac{\mathrm{number\ of\ documents}}{\mathrm{number\ of\ documents\ containing}\ t}
\end{align}
$

$
\begin{align}
\mathrm{tfidf}(t, d, D) = \mathrm{tf}(t, d) * \mathrm{idf}(t, D)
\end{align}
$

In [89]:
docs = {"intro": tokenizer.tokenize(kite_intro), "history": tokenizer.tokenize(kite_history)}

In [124]:
def tf(doc):
    count_doc = len(doc)
    count_terms = Counter(doc)
    return {term: count_term / count_doc for term, count_term in count_terms.items()}

tfs = {key: tf(doc) for key, doc in docs.items()}

In [125]:
from math import log
from functools import reduce

terms = set(reduce(lambda x, y: x + y, docs.values(), []))

def idf(docs, term):
    return log(len(docs) / len([True for doc in docs if term in doc]))

idfs = {term: idf(docs.values(), term) for term in terms}

In [126]:
def tfidf(tf, idf):
    return {term: tf_value * idfs[term] for term, tf_value in tf.items()}

tfidfs = {key: tfidf(tf, idf) for key, tf in tfs.items()}

In [106]:
tfidfs = {key: dict(sorted(tfidf.items(), key=lambda x: x[1], reverse=True)) for key, tfidf in tfidfs.items()}

### 3.4.2 Relevance ranking

前の章で単純な単語の出現数のカウントによるベクトルを作ったが、TF-IDFほど意味のあるものではなかった。
ここではまず、単語の出現回数を使っていたものをTF-IDFを使うようにしてみる。

In [127]:
from collections import OrderedDict
import copy

document_tfidf_vectors = []
harry_docs = [
    "The faster Harry got to the store, the faster and faster Harry would get home.",
    "Harry is hairy and faster than Jill.",
    "Jill is not as hairy as Harry."
]
doc_tokens = [sorted(tokenizer.tokenize(doc.lower())) for doc in harry_docs]
all_doc_tokens = sum(doc_tokens, [])
lexicon = sorted(set(all_doc_tokens))
zero_vector = OrderedDict((token, 0) for token in lexicon)
tfs = [tf(tokens) for tokens in doc_tokens]
idfs = {term: idf(doc_tokens, term) for term in all_doc_tokens}

for tf_dict in tfs:
    vec = copy.copy(zero_vector)
    for term, tf_value in tf_dict.items():
        vec[term] = tf_value * idfs[term]
    document_tfidf_vectors.append(vec)

[OrderedDict([(',', 0.06462425227459469),
              ('.', 0.0),
              ('and', 0.023850888712244965),
              ('as', 0),
              ('faster', 0.0715526661367349),
              ('get', 0.06462425227459469),
              ('got', 0.06462425227459469),
              ('hairy', 0),
              ('harry', 0.0),
              ('home', 0.06462425227459469),
              ('is', 0),
              ('jill', 0),
              ('not', 0),
              ('store', 0.06462425227459469),
              ('than', 0),
              ('the', 0.1938727568237841),
              ('to', 0.06462425227459469),
              ('would', 0.06462425227459469)]),
 OrderedDict([(',', 0),
              ('.', 0.0),
              ('and', 0.05068313851352055),
              ('as', 0),
              ('faster', 0.05068313851352055),
              ('get', 0),
              ('got', 0),
              ('hairy', 0.05068313851352055),
              ('harry', 0.0),
              ('home', 0),
              ('is'

In [147]:
query = "How long does it take to get to the store?"
query_vec = copy.copy(zero_vector)
query_tokens = tokenizer.tokenize(query.lower())

query_tf = tf(query_tokens)
query_idfs = {}
for term in query_tokens:
    try:
        query_idfs[term] = idf(doc_tokens, term)
    except:
        continue

for token, tf_value in query_tf.items():
    if query_idfs.get(token):
        query_vec[token] = tf_value * query_idfs[token]

In [148]:
cosine_sim(query_vec, document_tfidf_vectors[0])

0.6349617106273504

In [149]:
cosine_sim(query_vec, document_tfidf_vectors[1])

0.0

In [150]:
cosine_sim(query_vec, document_tfidf_vectors[2])

0.0

### 3.4.3 Tools

scikit-learn の[feature extractionライブラリ](https://scikit-learn.org/stable/modules/feature_extraction.html#feature-extraction)の中に[text向けのもの](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)があり、その一つとして[TF-IDFでベクトル化するモジュール](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting)が存在する。

In [153]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = harry_docs
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(corpus)
print(model.todense().round(2))

[[0.16 0.   0.48 0.21 0.21 0.   0.25 0.21 0.   0.   0.   0.21 0.   0.64
  0.21 0.21]
 [0.37 0.   0.37 0.   0.   0.37 0.29 0.   0.37 0.37 0.   0.   0.49 0.
  0.   0.  ]
 [0.   0.75 0.   0.   0.   0.29 0.22 0.   0.29 0.29 0.38 0.   0.   0.
  0.   0.  ]]


In [156]:
# get_feature_names() で各カラムがどの単語に対応しているのか確認できる。
vectorizer.get_feature_names()

['and',
 'as',
 'faster',
 'get',
 'got',
 'hairy',
 'harry',
 'home',
 'is',
 'jill',
 'not',
 'store',
 'than',
 'the',
 'to',
 'would']

### 3.4.4 Alternatives

TF-IDFマトリクスは数十年の間、情報検索の主力であり、研究者や企業によって検索結果の関連性の向上のために、IDF部分の最適化に時間が費やされてきた。その結果、多くの単語の頻度の重みを正規化および平準化する方法が生み出された。

参考: ["Word Embedding Past, Present and Future"](https://href.li/?https://youtu.be/edisTBOPBq0)

その中でも検索結果をランク付けする方法について、ストレートにTF-IDFのコサイン類似度の代替となる方法はOkapi BM25もしくはその最新のバリエーションであるBM25Fである。

### 3.4.5 Okapi BM25

q_idf * dot(q_tf, d_tf[i]) * 1.5 / (dot(q_tf, d_tf[i]) + 0.25 + 0.75 * d_num_words[i] / d_num_words.mean())

## まとめ
- Term frequencyは最も重要であり、最も意味のある単語に適切な重みを与えるために、IDF（Inverse document frequency）で重み付けする必要がある。
- ジップの法則は、単語、文字、人など、あらゆる種類のものの頻度を予測するのに役立ちます。
- TF-IDF単語ドキュメントマトリックスの行は、それらの個々の単語の意味のベクトル表現として使用して、単語セマンティクスのベクトル空間モデルを作成できます。
- ベクトル間の「オーバーラップ」の量であるコサイン距離は、正規化されたベクトルの要素を乗算し、それらの積を合計するだけで効率的に計算できます
- コサイン距離は、ほとんどの自然言語ベクトル表現の頼りになる類似性スコアです。